In [64]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/System-Threat-Forecaster/sample_submission.csv
/kaggle/input/System-Threat-Forecaster/train.csv
/kaggle/input/System-Threat-Forecaster/test.csv


In [65]:
train = pd.read_csv("/kaggle/input/System-Threat-Forecaster/train.csv")


In [66]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler


In [67]:
train.columns

Index(['MachineID', 'ProductName', 'EngineVersion', 'AppVersion',
       'SignatureVersion', 'IsBetaUser', 'RealTimeProtectionState',
       'IsPassiveModeEnabled', 'AntivirusConfigID',
       'NumAntivirusProductsInstalled', 'NumAntivirusProductsEnabled',
       'HasTpm', 'CountryID', 'CityID', 'GeoRegionID', 'LocaleEnglishNameID',
       'PlatformType', 'Processor', 'OSVersion', 'OSBuildNumber',
       'OSProductSuite', 'OsPlatformSubRelease', 'OSBuildLab',
       'SKUEditionName', 'IsSystemProtected', 'AutoSampleSubmissionEnabled',
       'SMode', 'IEVersionID', 'FirewallEnabled', 'EnableLUA',
       'MDC2FormFactor', 'DeviceFamily', 'OEMNameID', 'OEMModelID',
       'ProcessorCoreCount', 'ProcessorManufacturerID', 'ProcessorModelID',
       'PrimaryDiskCapacityMB', 'PrimaryDiskType', 'SystemVolumeCapacityMB',
       'HasOpticalDiskDrive', 'TotalPhysicalRAMMB', 'ChassisType',
       'PrimaryDisplayDiagonalInches', 'PrimaryDisplayResolutionHorizontal',
       'PrimaryDisplayResolutio

In [68]:
train.shape

(100000, 76)

There are 1 lakh rows and 76 columns are there in the train dataset and there are ten thousand rows and 75 columns are there in the test dataset.

In [69]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 76 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   MachineID                           100000 non-null  object 
 1   ProductName                         100000 non-null  object 
 2   EngineVersion                       100000 non-null  object 
 3   AppVersion                          100000 non-null  object 
 4   SignatureVersion                    100000 non-null  object 
 5   IsBetaUser                          100000 non-null  int64  
 6   RealTimeProtectionState             99934 non-null   float64
 7   IsPassiveModeEnabled                100000 non-null  int64  
 8   AntivirusConfigID                   99924 non-null   float64
 9   NumAntivirusProductsInstalled       99924 non-null   float64
 10  NumAntivirusProductsEnabled         99924 non-null   float64
 11  HasTpm                     

In [70]:
train.head()

,MachineID,ProductName,EngineVersion,AppVersion,SignatureVersion,IsBetaUser,RealTimeProtectionState,IsPassiveModeEnabled,AntivirusConfigID,NumAntivirusProductsInstalled,NumAntivirusProductsEnabled,HasTpm,CountryID,CityID,GeoRegionID,LocaleEnglishNameID,PlatformType,Processor,OSVersion,OSBuildNumber,OSProductSuite,OsPlatformSubRelease,OSBuildLab,SKUEditionName,IsSystemProtected,AutoSampleSubmissionEnabled,SMode,IEVersionID,FirewallEnabled,EnableLUA,MDC2FormFactor,DeviceFamily,OEMNameID,OEMModelID,ProcessorCoreCount,ProcessorManufacturerID,ProcessorModelID,PrimaryDiskCapacityMB,PrimaryDiskType,SystemVolumeCapacityMB,HasOpticalDiskDrive,TotalPhysicalRAMMB,ChassisType,PrimaryDisplayDiagonalInches,PrimaryDisplayResolutionHorizontal,PrimaryDisplayResolutionVertical,PowerPlatformRole,InternalBatteryNumberOfCharges,NumericOSVersion,OSArchitecture,OSBranch,OSBuildNumberOnly,OSBuildRevisionOnly,OSEdition,OSSkuFriendlyName,OSInstallType,OSInstallLanguageID,OSUILocaleID,AutoUpdateOptionsName,IsPortableOS,OSGenuineState,LicenseActivationChannel,IsFlightsDisabled,FlightRing,FirmwareManufacturerID,FirmwareVersionID,IsSecureBootEnabled,IsVirtualDevice,IsTouchEnabled,IsPenCapable,IsAlwaysOnAlwaysConnectedCapable,IsGamer,RegionIdentifier,DateAS,DateOS,target
0,f541bae429089117c4aac39c90dd3416,win8defender,1.1.15200.1,4.18.1807.18075,1.275.1003.0,0,7.0,0,53447.0,1.0,1.0,1,51,120232.0,98.0,103,windows10,x86,10.0.0.0,14393,768,rs1,14393.2214.x86fre.rs1_release_1.180402-1758,Home,1.0,0,0.0,98.0,1.0,1.0,SmallTablet,Windows.Desktop,561.0,330367.0,4.0,5.0,1850.0,15028.0,SSD,14348.0,0,1024.0,Notebook,8.0,800.0,1280.0,Slate,1.420000e+02,10.0.14393.2214,x86,rs1_release,14393,2214,Core,CORE,Update,5.0,26,UNKNOWN,0,IS_GENUINE,Retail,0.0,Retail,513.0,21964.0,0,0.0,1,0,1.0,0.0,6.0,2018-09-10 10:11:00,2018-04-17,0
1,dc2b14d9ce3a0ce4050bb640190f2ca5,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1465.0,0,7.0,0,53447.0,1.0,1.0,1,141,112854.0,167.0,227,windows10,x64,10.0.0.0,17134,256,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,1.0,0,0.0,137.0,1.0,1.0,AllInOne,Windows.Desktop,2668.0,25212.0,4.0,5.0,2407.0,953869.0,HDD,952592.0,1,4096.0,AllinOne,19.4,1600.0,900.0,Desktop,4.294967e+09,10.0.17134.228,amd64,rs4_release,17134,228,Professional,PROFESSIONAL,UUPUpgrade,9.0,34,FullAuto,0,IS_GENUINE,OEM:DM,0.0,Retail,628.0,44548.0,1,0.0,0,0,0.0,0.0,10.0,2018-08-16 00:01:00,2018-08-14,1
2,fd20c5f010e9c5f91ad1c6b3e0da68a0,win8defender,1.1.15200.1,4.18.1807.18075,1.275.1546.0,0,7.0,0,53447.0,1.0,1.0,1,51,41759.0,98.0,103,windows10,x64,10.0.0.0,17134,768,rs4,17134.1.amd64fre.rs4_release.180410-1804,Home,1.0,0,0.0,137.0,1.0,1.0,Desktop,Windows.Desktop,3035.0,263666.0,4.0,5.0,2719.0,228936.0,SSD,228321.0,1,8192.0,Desktop,24.0,1920.0,1080.0,Desktop,4.294967e+09,10.0.17134.285,amd64,rs4_release,17134,285,Core,CORE,Reset,5.0,26,FullAuto,0,IS_GENUINE,OEM:NONSLP,0.0,Retail,142.0,9414.0,0,0.0,0,0,0.0,1.0,6.0,2018-09-20 23:20:00,2018-09-11,1
3,38711eae85eb77a72ec5dfdf27eb2a76,win8defender,1.1.15200.1,4.12.17007.18011,1.275.1141.0,0,7.0,0,46413.0,2.0,1.0,1,68,19507.0,276.0,74,windows10,x64,10.0.0.0,15063,768,rs2,15063.0.amd64fre.rs2_release.170317-1834,Home,1.0,0,0.0,108.0,1.0,1.0,Notebook,Windows.Desktop,2102.0,242491.0,4.0,5.0,3410.0,1907729.0,HDD,1890776.0,0,8192.0,Notebook,15.5,1366.0,768.0,Mobile,0.000000e+00,10.0.15063.850,amd64,rs2_release,15063,850,Core,CORE,Upgrade,7.0,30,UNKNOWN,0,IS_GENUINE,OEM:DM,0.0,Retail,554.0,33060.0,1,0.0,0,0,0.0,0.0,12.0,2018-09-14 00:32:00,2018-01-03,1
4,32607c9a543a9214e2c7e45800ed4849,win8defender,1.1.15200.1,4.13.17134.228,1.275.1283.0,0,7.0,0,40466.0,2.0,1.0,1,43,117801.0,53.0,42,windows10,x86,10.0.0.0,17134,256,rs4,17134.1.x86fre.rs4_release.180410-1804,Pro,1.0,0,0.0,137.0,1.0,1.0,Desktop,Windows.Desktop,2668.0,257309.0,2.0,5.0,4322.0,305245.0,HDD,52804.0,0,2048.0,Desktop,20.0,1600.0,900.0,Desktop,4.294967e+09,10.0.17134.285,x86,rs4_release,17134,285,Professional,PROFESSIONAL,UUPUpgrade,37.0,158,FullAuto,0,IS_GENUINE,Retail,0.0,Retail,628.0

In [71]:
train.tail()

,MachineID,ProductName,EngineVersion,AppVersion,SignatureVersion,IsBetaUser,RealTimeProtectionState,IsPassiveModeEnabled,AntivirusConfigID,NumAntivirusProductsInstalled,NumAntivirusProductsEnabled,HasTpm,CountryID,CityID,GeoRegionID,LocaleEnglishNameID,PlatformType,Processor,OSVersion,OSBuildNumber,OSProductSuite,OsPlatformSubRelease,OSBuildLab,SKUEditionName,IsSystemProtected,AutoSampleSubmissionEnabled,SMode,IEVersionID,FirewallEnabled,EnableLUA,MDC2FormFactor,DeviceFamily,OEMNameID,OEMModelID,ProcessorCoreCount,ProcessorManufacturerID,ProcessorModelID,PrimaryDiskCapacityMB,PrimaryDiskType,SystemVolumeCapacityMB,HasOpticalDiskDrive,TotalPhysicalRAMMB,ChassisType,PrimaryDisplayDiagonalInches,PrimaryDisplayResolutionHorizontal,PrimaryDisplayResolutionVertical,PowerPlatformRole,InternalBatteryNumberOfCharges,NumericOSVersion,OSArchitecture,OSBranch,OSBuildNumberOnly,OSBuildRevisionOnly,OSEdition,OSSkuFriendlyName,OSInstallType,OSInstallLanguageID,OSUILocaleID,AutoUpdateOptionsName,IsPortableOS,OSGenuineState,LicenseActivationChannel,IsFlightsDisabled,FlightRing,FirmwareManufacturerID,FirmwareVersionID,IsSecureBootEnabled,IsVirtualDevice,IsTouchEnabled,IsPenCapable,IsAlwaysOnAlwaysConnectedCapable,IsGamer,RegionIdentifier,DateAS,DateOS,target
99995,a378b2196a7a1d8ee0d4f989ee46730e,win8defender,1.1.15200.1,4.18.1807.18075,1.275.1582.0,0,7.0,0,68585.0,2.0,1.0,1,43,71228.0,53.0,42,windows10,x64,10.0.0.0,16299,256,rs3,16299.431.amd64fre.rs3_release_svc_escrow.1805...,Pro,1.0,0,0.0,117.0,1.0,1.0,Desktop,Windows.Desktop,4909.0,317701.0,4.0,5.0,2615.0,122104.0,SSD,121488.0,0,8192.0,Desktop,23.8,1920.0,1080.0,Desktop,4.294967e+09,10.0.17134.319,amd64,rs4_release,17134,319,Professional,PROFESSIONAL,UUPUpgrade,37.0,158,FullAuto,0,IS_GENUINE,Retail,0.0,Retail,142.0,25740.0,0,0.0,0,0,0.0,1.0,7.0,2018-09-21 00:59:00,2018-09-20,0
99996,7f4433ca05d24b9bbdd3932f8b05e61f,win8defender,1.1.15100.1,4.18.1806.18062,1.273.1156.0,0,7.0,0,11280.0,2.0,1.0,1,137,137066.0,160.0,74,windows10,x64,10.0.0.0,17134,256,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,1.0,0,0.0,137.0,1.0,1.0,Notebook,Windows.Desktop,1443.0,260853.0,4.0,5.0,2529.0,476940.0,HDD,475637.0,0,4096.0,Laptop,13.2,1366.0,768.0,Mobile,0.000000e+00,10.0.17134.165,amd64,rs4_release,17134,165,Professional,PROFESSIONAL,UUPUpgrade,8.0,31,FullAuto,0,IS_GENUINE,Retail,0.0,Retail,355.0,20900.0,0,0.0,0,0,0.0,0.0,3.0,2018-08-10 07:23:00,2018-07-10,0
99997,d6852dfff534e03d5ddfd3c7875ab072,win8defender,1.1.15200.1,4.18.1807.18075,1.275.209.0,0,7.0,0,53447.0,1.0,1.0,1,142,74924.0,157.0,68,windows10,x64,10.0.0.0,16299,256,rs3,16299.15.amd64fre.rs3_release.170928-1534,Pro,1.0,0,0.0,111.0,1.0,1.0,Desktop,Windows.Desktop,2102.0,250496.0,4.0,5.0,2719.0,953869.0,HDD,532690.0,0,4096.0,Desktop,18.5,1366.0,768.0,Desktop,4.294967e+09,10.0.16299.309,amd64,rs3_release,16299,309,Professional,PROFESSIONAL,Upgrade,8.0,31,Notify,0,IS_GENUINE,OEM:DM,0.0,Retail,486.0,51605.0,1,0.0,0,0,0.0,1.0,1.0,2018-08-26 23:51:00,2018-03-13,1
99998,70a8dc682128c9ff78a3de5aea9dac74,win8defender,1.1.14901.4,4.16.17656.18052,1.269.641.0,0,7.0,0,53447.0,1.0,1.0,1,139,87570.0,158.0,74,windows10,x64,10.0.0.0,17134,256,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,1.0,0,0.0,137.0,1.0,1.0,Notebook,Windows.Desktop,3150.0,313499.0,4.0,5.0,3032.0,976762.0,SSD,497353.0,0,16384.0,Laptop,13.5,3000.0,2000.0,Slate,4.500000e+01,10.0.17134.48,amd64,rs4_release,17134,48,Professional,PROFESSIONAL,UUPUpgrade,8.0,31,Notify,0,IS_GENUINE,OEM:DM,0.0,Unknown,677.0,18669.0,1,0.0,1,1,1.0,0.0,1.0,2018-06-04 17:13:00,2018-05-08,0
99999,ad9c385b53f2ca788933a39603b84eda,win8defender,1.1.15000.2,4.18.1806.18062,1.271.1003.0,0,7.0,0,7945.0,2.0,1.0,1,100,4110.0,224.0,75,windows10,x64,10.0.0.0,17134,768,rs4,17134.1.amd64fre.rs4_release.180410-1804,Home,1.0,0,0.0,137.0,1.0,1.0,Notebook,Windows.Desktop,1443.0,256682.0,4.0,5.0,3026.0,953869.0,HDD,937373.0,0,8192.0,Notebook,15.5,1366.0,768.0,Mobile,0.000000e+00,10.0.17134.228,amd64,rs4_release,17134,228,Core,CORE,UUPUp

# Explanatory Data Analysis

In [72]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
train.nunique()

MachineID                             99835
ProductName                               2
EngineVersion                            37
AppVersion                               69
SignatureVersion                       2735
IsBetaUser                                1
RealTimeProtectionState                   6
IsPassiveModeEnabled                      2
AntivirusConfigID                      1976
NumAntivirusProductsInstalled             5
NumAntivirusProductsEnabled               5
HasTpm                                    2
CountryID                               220
CityID                                16047
GeoRegionID                             239
LocaleEnglishNameID                     185
PlatformType                              4
Processor                                 3
OSVersion                                 7
OSBuildNumber                            24
OSProductSuite                            7
OsPlatformSubRelease                      9
OSBuildLab                      

In [73]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
train.isnull().sum()

MachineID                               0
ProductName                             0
EngineVersion                           0
AppVersion                              0
SignatureVersion                        0
IsBetaUser                              0
RealTimeProtectionState                66
IsPassiveModeEnabled                    0
AntivirusConfigID                      76
NumAntivirusProductsInstalled          76
NumAntivirusProductsEnabled            76
HasTpm                                  0
CountryID                               0
CityID                                623
GeoRegionID                             0
LocaleEnglishNameID                     0
PlatformType                            0
Processor                               0
OSVersion                               0
OSBuildNumber                           0
OSProductSuite                          0
OsPlatformSubRelease                    0
OSBuildLab                              0
SKUEditionName                    

In [74]:
no_missing_cols = train.columns[train.isnull().sum() == 0]

# Print the column names
print("Columns with no missing values:")
print(no_missing_cols)

# Count the number of columns with no missing values
print("\nNumber of columns with no missing values:", len(no_missing_cols))

Columns with no missing values:
Index(['MachineID', 'ProductName', 'EngineVersion', 'AppVersion',
       'SignatureVersion', 'IsBetaUser', 'IsPassiveModeEnabled', 'HasTpm',
       'CountryID', 'GeoRegionID', 'LocaleEnglishNameID', 'PlatformType',
       'Processor', 'OSVersion', 'OSBuildNumber', 'OSProductSuite',
       'OsPlatformSubRelease', 'OSBuildLab', 'SKUEditionName',
       'AutoSampleSubmissionEnabled', 'MDC2FormFactor', 'DeviceFamily',
       'HasOpticalDiskDrive', 'PowerPlatformRole', 'NumericOSVersion',
       'OSArchitecture', 'OSBranch', 'OSBuildNumberOnly',
       'OSBuildRevisionOnly', 'OSEdition', 'OSSkuFriendlyName',
       'OSInstallType', 'OSUILocaleID', 'AutoUpdateOptionsName',
       'IsPortableOS', 'OSGenuineState', 'LicenseActivationChannel',
       'FlightRing', 'IsSecureBootEnabled', 'IsTouchEnabled', 'IsPenCapable',
       'DateAS', 'target'],
      dtype='object')

Number of columns with no missing values: 43


There are 43 columns having missing values in the whole dataset.

In [75]:
one_unique_value = train.columns[train.nunique()==1]
print(one_unique_value)

Index(['IsBetaUser', 'AutoSampleSubmissionEnabled', 'IsFlightsDisabled'], dtype='object')


These columns contain only one unique value which doesn't makes much sense so i remove these columns.

# Preprocessing

In [76]:
train['ProductName'] = train['ProductName'].replace({'win8defender': 1, 'mse': 0})
train['DeviceFamily'] = train['DeviceFamily'].replace({'Windows.Server':0, 'Windows.Desktop':1})

In [77]:
columns_to_fill = ['CityID', 'NumAntivirusProductsEnabled']
train[columns_to_fill] = train[columns_to_fill].fillna(0)

In [78]:
train['PrimaryDiskType'] = train['PrimaryDiskType'].replace(['UNKNOWN','Unknown','Unspecified'], 'Unknown')
train['PrimaryDiskType'] = train['PrimaryDiskType'].fillna('Unknown')



'DateOS','DateAS','FirmwareVersionID' , 'SystemVolumeCapacityMB' , 'OEMModelID' , 'CityID' , 'MachineID'. They contain lot of unique values





In [79]:
train.shape

(100000, 76)

In [80]:
train = train.drop(columns= ['IsBetaUser', 'IsFlightsDisabled','DateOS','FirmwareVersionID' , 'SystemVolumeCapacityMB' , 'OEMModelID' , 'CityID' , 'MachineID'], errors='ignore')

In [81]:
train.shape

(100000, 68)

In [82]:
train.columns

Index(['ProductName', 'EngineVersion', 'AppVersion', 'SignatureVersion',
       'RealTimeProtectionState', 'IsPassiveModeEnabled', 'AntivirusConfigID',
       'NumAntivirusProductsInstalled', 'NumAntivirusProductsEnabled',
       'HasTpm', 'CountryID', 'GeoRegionID', 'LocaleEnglishNameID',
       'PlatformType', 'Processor', 'OSVersion', 'OSBuildNumber',
       'OSProductSuite', 'OsPlatformSubRelease', 'OSBuildLab',
       'SKUEditionName', 'IsSystemProtected', 'AutoSampleSubmissionEnabled',
       'SMode', 'IEVersionID', 'FirewallEnabled', 'EnableLUA',
       'MDC2FormFactor', 'DeviceFamily', 'OEMNameID', 'ProcessorCoreCount',
       'ProcessorManufacturerID', 'ProcessorModelID', 'PrimaryDiskCapacityMB',
       'PrimaryDiskType', 'HasOpticalDiskDrive', 'TotalPhysicalRAMMB',
       'ChassisType', 'PrimaryDisplayDiagonalInches',
       'PrimaryDisplayResolutionHorizontal',
       'PrimaryDisplayResolutionVertical', 'PowerPlatformRole',
       'InternalBatteryNumberOfCharges', 'NumericOS

In [83]:
numeric_cols = train.select_dtypes(include=['int']).columns.tolist()
object_cols = train.select_dtypes(include=['object']).columns.tolist()
float_cols = train.select_dtypes(include=['float64']).columns.tolist()

print("\nNumeric:", len(numeric_cols), numeric_cols)
print("\nObject:", len(object_cols), object_cols)
print("\nFloat:", len(float_cols), float_cols)


Numeric: 18 ['ProductName', 'IsPassiveModeEnabled', 'HasTpm', 'CountryID', 'LocaleEnglishNameID', 'OSBuildNumber', 'OSProductSuite', 'AutoSampleSubmissionEnabled', 'DeviceFamily', 'HasOpticalDiskDrive', 'OSBuildNumberOnly', 'OSBuildRevisionOnly', 'OSUILocaleID', 'IsPortableOS', 'IsSecureBootEnabled', 'IsTouchEnabled', 'IsPenCapable', 'target']

Object: 24 ['EngineVersion', 'AppVersion', 'SignatureVersion', 'PlatformType', 'Processor', 'OSVersion', 'OsPlatformSubRelease', 'OSBuildLab', 'SKUEditionName', 'MDC2FormFactor', 'PrimaryDiskType', 'ChassisType', 'PowerPlatformRole', 'NumericOSVersion', 'OSArchitecture', 'OSBranch', 'OSEdition', 'OSSkuFriendlyName', 'OSInstallType', 'AutoUpdateOptionsName', 'OSGenuineState', 'LicenseActivationChannel', 'FlightRing', 'DateAS']

Float: 26 ['RealTimeProtectionState', 'AntivirusConfigID', 'NumAntivirusProductsInstalled', 'NumAntivirusProductsEnabled', 'GeoRegionID', 'IsSystemProtected', 'SMode', 'IEVersionID', 'FirewallEnabled', 'EnableLUA', 'OEMNa

# Simple Imputer

In [84]:
train = train.replace({pd.NA: np.nan, '': np.nan, None: np.nan})
all_columns = {"numeric": numeric_cols, "object": object_cols, "float": float_cols}
mode_imputer = SimpleImputer(strategy='most_frequent')

In [85]:
# Impute each column type
for col_type, columns in all_columns.items():
    if columns:  # Ensure there are columns to process
        train[columns] = pd.DataFrame(
            mode_imputer.fit_transform(train[columns]),
            columns=columns,
            index=train.index
        )

In [86]:
for col in all_columns["numeric"]:
    train[col] = train[col].astype(int)
for col in all_columns["float"]:
    train[col] = train[col].astype(float)
for col in all_columns["object"]:
    train[col] = train[col].astype(str)
print(train.isnull().sum())

ProductName                           0
EngineVersion                         0
AppVersion                            0
SignatureVersion                      0
RealTimeProtectionState               0
IsPassiveModeEnabled                  0
AntivirusConfigID                     0
NumAntivirusProductsInstalled         0
NumAntivirusProductsEnabled           0
HasTpm                                0
CountryID                             0
GeoRegionID                           0
LocaleEnglishNameID                   0
PlatformType                          0
Processor                             0
OSVersion                             0
OSBuildNumber                         0
OSProductSuite                        0
OsPlatformSubRelease                  0
OSBuildLab                            0
SKUEditionName                        0
IsSystemProtected                     0
AutoSampleSubmissionEnabled           0
SMode                                 0
IEVersionID                           0


In [87]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
train.nunique()

ProductName                              2
EngineVersion                           37
AppVersion                              69
SignatureVersion                      2735
RealTimeProtectionState                  6
IsPassiveModeEnabled                     2
AntivirusConfigID                     1976
NumAntivirusProductsInstalled            5
NumAntivirusProductsEnabled              5
HasTpm                                   2
CountryID                              220
GeoRegionID                            239
LocaleEnglishNameID                    185
PlatformType                             4
Processor                                3
OSVersion                                7
OSBuildNumber                           24
OSProductSuite                           7
OsPlatformSubRelease                     9
OSBuildLab                             281
SKUEditionName                           8
IsSystemProtected                        2
AutoSampleSubmissionEnabled              1
SMode      

In [88]:
unique_counts = train.nunique()
binary_cols = unique_counts[unique_counts == 2].index
binary = train[binary_cols]
print(binary.head())


   ProductName  IsPassiveModeEnabled  HasTpm  IsSystemProtected  SMode  \
0            1                     0       1                1.0    0.0   
1            1                     0       1                1.0    0.0   
2            1                     0       1                1.0    0.0   
3            1                     0       1                1.0    0.0   
4            1                     0       1                1.0    0.0   

   FirewallEnabled  DeviceFamily  HasOpticalDiskDrive  IsPortableOS  \
0              1.0             1                    0             0   
1              1.0             1                    1             0   
2              1.0             1                    1             0   
3              1.0             1                    0             0   
4              1.0             1                    0             0   

   IsSecureBootEnabled  IsVirtualDevice  IsTouchEnabled  IsPenCapable  \
0                    0              0.0               1

In [89]:
# Select columns with int64 dtype
int_cols = train.select_dtypes(include=['int64']).columns.tolist()

# Convert int64 columns to float64
for col in int_cols:
    train[col] = train[col].astype('float64', errors='ignore')

# Print the updated info
print(train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 68 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   ProductName                         100000 non-null  float64
 1   EngineVersion                       100000 non-null  object 
 2   AppVersion                          100000 non-null  object 
 3   SignatureVersion                    100000 non-null  object 
 4   RealTimeProtectionState             100000 non-null  float64
 5   IsPassiveModeEnabled                100000 non-null  float64
 6   AntivirusConfigID                   100000 non-null  float64
 7   NumAntivirusProductsInstalled       100000 non-null  float64
 8   NumAntivirusProductsEnabled         100000 non-null  float64
 9   HasTpm                              100000 non-null  float64
 10  CountryID                           100000 non-null  float64
 11  GeoRegionID                

In [90]:
# Select columns with float64 dtype
float64_cols = train.select_dtypes(include=['float64','int']).columns.tolist()

# Iterate through float64 columns (excluding 'target') and calculate the correlation with 'target'
correlations = {}
for col in float64_cols:
    if col != 'target':
        correlation = train[col].corr(train['target'])
        correlations[col] = abs(correlation) #finding absolute value
# Sort correlations by absolute value in descending order
sorted_correlations = dict(sorted(correlations.items(), key=lambda item: item[1], reverse=True))

# Print the sorted correlations
for col, corr in sorted_correlations.items():
    print(f"Correlation between '{col}' and 'target': {corr}")

Correlation between 'NumAntivirusProductsInstalled' and 'target': 0.15112661954826917
Correlation between 'AntivirusConfigID' and 'target': 0.11814587405377451
Correlation between 'AutoSampleSubmissionEnabled' and 'target': nan
Correlation between 'TotalPhysicalRAMMB' and 'target': 0.06568266537530583
Correlation between 'IsAlwaysOnAlwaysConnectedCapable' and 'target': 0.06474655260779519
Correlation between 'IsSystemProtected' and 'target': 0.06231802390060379
Correlation between 'IsGamer' and 'target': 0.06090142129232548
Correlation between 'ProcessorCoreCount' and 'target': 0.056744265347587565
Correlation between 'PrimaryDiskCapacityMB' and 'target': 0.04908678016770174
Correlation between 'RealTimeProtectionState' and 'target': 0.04897668399321169
Correlation between 'IsTouchEnabled' and 'target': 0.04614230100840552
Correlation between 'IsPassiveModeEnabled' and 'target': 0.04199661646513612
Correlation between 'IsVirtualDevice' and 'target': 0.039781886788428714
Correlation bet

/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


In [91]:
for col in object_cols:
    if col != 'target':
        # Calculate the number of unique values in the column
        num_unique_values = train[col].nunique()

        print(f"Column: {col}")
        if num_unique_values < 50:
          print(train[col].value_counts())

        #Group by the object column and calculate the mean of the target variable for each group
        relationship = train.groupby(col)['target'].mean()
        print(f"Relationship with 'target':\n{relationship}")
        print("-" * 20)

Column: EngineVersion
EngineVersion
1.1.15200.1    45219
1.1.15100.1    44017
1.1.15000.2     3169
1.1.14901.4     2597
1.1.14800.3     1719
1.1.15300.6      650
1.1.14700.5      593
1.1.14600.4      569
1.1.14104.0      328
1.1.15300.5      255
1.1.13504.0      212
1.1.14500.5      124
1.1.14405.2       82
1.1.14306.0       56
1.1.14202.0       50
1.1.14003.0       34
1.1.13804.0       33
1.1.13903.0       28
1.1.13303.0       27
1.1.13601.0       26
1.1.13407.0       23
1.1.14901.3       22
1.1.15000.1       22
1.1.13103.0       20
1.1.14305.0       17
1.1.14700.4       16
1.1.12902.0       15
1.1.13701.0       15
1.1.13704.0       15
1.1.13202.0       14
1.1.13000.0       10
1.1.14700.3        9
1.1.14800.1        7
1.1.14500.2        3
1.1.12805.0        2
1.1.13802.0        1
1.1.13803.0        1
Name: count, dtype: int64
Relationship with 'target':
EngineVersion
1.1.12805.0    0.000000
1.1.12902.0    0.333333
1.1.13000.0    0.000000
1.1.13103.0    0.400000
1.1.13202.0    0.285714

In [92]:
correlation_results = {}

# Iterate through object columns and calculate the relationship with 'target'
for col in object_cols:
    if col != 'target':

        relationship = train.groupby(col)['target'].mean()
        correlation_results[col] = relationship
top_9_object_features = sorted(correlation_results.items(), key=lambda x: x[1].std(), reverse=True)
for feature, correlation in top_9_object_features:
  print(f"Feature: {feature}, Correlation: {correlation.std()}")

Feature: OSVersion, Correlation: 0.3762530804643262
Feature: SignatureVersion, Correlation: 0.3007191744051268
Feature: DateAS, Correlation: 0.30021787979218123
Feature: OSEdition, Correlation: 0.2672813542078498
Feature: OSBuildLab, Correlation: 0.2648752202781123
Feature: OSSkuFriendlyName, Correlation: 0.25284281580355317
Feature: PowerPlatformRole, Correlation: 0.2130039354168778
Feature: AppVersion, Correlation: 0.20947959651458817
Feature: NumericOSVersion, Correlation: 0.19302564451467785
Feature: ChassisType, Correlation: 0.18685748118627132
Feature: OSArchitecture, Correlation: 0.15915074665989765
Feature: Processor, Correlation: 0.15915029042516268
Feature: EngineVersion, Correlation: 0.1546545109270554
Feature: LicenseActivationChannel, Correlation: 0.1539970277622421
Feature: SKUEditionName, Correlation: 0.12831866129408556
Feature: MDC2FormFactor, Correlation: 0.12419513532657754
Feature: PlatformType, Correlation: 0.10940270854692458
Feature: AutoUpdateOptionsName, Correl

# Train-Test Split

In [93]:
# Selected numerical and categorical features based on correlation analysis
numerical_col = [
    'NumAntivirusProductsInstalled', 'AntivirusConfigID', 'AutoSampleSubmissionEnabled',
    'TotalPhysicalRAMMB', 'IsAlwaysOnAlwaysConnectedCapable', 'IsSystemProtected',
    'IsGamer', 'ProcessorCoreCount', 'PrimaryDiskCapacityMB', 'RealTimeProtectionState',
    'IsTouchEnabled', 'IsPassiveModeEnabled', 'IsVirtualDevice', 'NumAntivirusProductsEnabled',
    'OSBuildNumberOnly', 'PrimaryDisplayDiagonalInches', 'OSBuildNumber',
    'PrimaryDisplayResolutionHorizontal', 'IEVersionID', 'HasOpticalDiskDrive',
    'ProcessorModelID', 'OSProductSuite', 'IsPenCapable', 'FirmwareManufacturerID',
    'RegionIdentifier', 'InternalBatteryNumberOfCharges', 'LocaleEnglishNameID',
    'SMode', 'OEMNameID', 'PrimaryDisplayResolutionVertical', 'FirewallEnabled',
    'OSBuildRevisionOnly', 'DeviceFamily', 'HasTpm', 'IsPortableOS', 'ProductName',
    'CountryID', 'OSInstallLanguageID', 'ProcessorManufacturerID', 'GeoRegionID',
    'IsSecureBootEnabled', 'OSUILocaleID', 'EnableLUA', 'target'
]


categorical_col = [
    'OSVersion', 'SignatureVersion', 'OSEdition', 'OSBuildLab','DateAS',
    'OSSkuFriendlyName', 'PowerPlatformRole', 'AppVersion', 'NumericOSVersion',
    'ChassisType', 'OSArchitecture', 'Processor', 'EngineVersion',
    'LicenseActivationChannel', 'SKUEditionName', 'MDC2FormFactor',
    'PlatformType', 'AutoUpdateOptionsName', 'OSGenuineState', 'FlightRing',
    'OSBranch', 'PrimaryDiskType', 'OSInstallType', 'OsPlatformSubRelease'
]


In [94]:
print(len(numerical_col))
print(len(categorical_col))

44
24


In [95]:
# Combine the numerical and categorical columns into a single list
selected_col = numerical_col + categorical_col

# Select only the specified columns and drop all others
train = train[selected_col]

# Verify the resulting DataFrame
print("Columns remaining in the DataFrame:", train.columns.tolist())
print(len(train.columns))

Columns remaining in the DataFrame: ['NumAntivirusProductsInstalled', 'AntivirusConfigID', 'AutoSampleSubmissionEnabled', 'TotalPhysicalRAMMB', 'IsAlwaysOnAlwaysConnectedCapable', 'IsSystemProtected', 'IsGamer', 'ProcessorCoreCount', 'PrimaryDiskCapacityMB', 'RealTimeProtectionState', 'IsTouchEnabled', 'IsPassiveModeEnabled', 'IsVirtualDevice', 'NumAntivirusProductsEnabled', 'OSBuildNumberOnly', 'PrimaryDisplayDiagonalInches', 'OSBuildNumber', 'PrimaryDisplayResolutionHorizontal', 'IEVersionID', 'HasOpticalDiskDrive', 'ProcessorModelID', 'OSProductSuite', 'IsPenCapable', 'FirmwareManufacturerID', 'RegionIdentifier', 'InternalBatteryNumberOfCharges', 'LocaleEnglishNameID', 'SMode', 'OEMNameID', 'PrimaryDisplayResolutionVertical', 'FirewallEnabled', 'OSBuildRevisionOnly', 'DeviceFamily', 'HasTpm', 'IsPortableOS', 'ProductName', 'CountryID', 'OSInstallLanguageID', 'ProcessorManufacturerID', 'GeoRegionID', 'IsSecureBootEnabled', 'OSUILocaleID', 'EnableLUA', 'target', 'OSVersion', 'Signatur

In [96]:
y= train.target.copy()
x= train.drop('target', axis=1).copy()

In [97]:
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val = train_test_split(x, y, test_size=0.15, random_state=1)

In [98]:
X_train.shape


(85000, 67)

In [99]:
X_val.shape

(15000, 67)

In [100]:
y_train.shape

(85000,)

In [101]:
y_val.shape

(15000,)

# Pipeline

In [102]:
from sklearn.compose import make_column_selector as mcs
num_sel = mcs(dtype_exclude='object')
cat_sel = mcs(dtype_include='object')

In [103]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, FunctionTransformer,OneHotEncoder
from sklearn.compose import ColumnTransformer

# Create numerical and categorical pipelines
num_pipeline = Pipeline([
    ('scaler', StandardScaler())
])

# Custom transformer to convert float outputs to integers
def convert_to_int(X):
    return X.astype(np.int64)  # Convert to integer

cat_pipeline = Pipeline([
    ('ordinal', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)),
    ('convert_to_int', FunctionTransformer(convert_to_int))  # Convert floats to integers
])

# Combine numerical and categorical pipelines using ColumnTransformer
preprocessor = ColumnTransformer([
    ('numerical', num_pipeline, num_sel(X_train)),
    ('categorical', cat_pipeline, cat_sel(X_train))
])

# Create and fit the complete pipeline
pipe = Pipeline([('preprocessor', preprocessor)])
pipe.fit(X_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['NumAntivirusProductsInstalled',
                                                   'AntivirusConfigID',
                                                   'AutoSampleSubmissionEnabled',
                                                   'TotalPhysicalRAMMB',
                                                   'IsAlwaysOnAlwaysConnectedCapable',
                                                   'IsSystemProtected',
                                                   'IsGamer',
                                                   'ProcessorCoreCount',
                                                   'PrimaryDiskCapacityMB',
                                                   'RealTimeProt...
                                                   'OSEdition', 'OSBuildLab',
                                                   'DateAS',
                                                   'OSSkuFriendlyName',
                                                   'PowerPlatformRole',
                                                   'AppVersion',
                                                   'NumericOSVersion',
                                                   'ChassisType',
                                                   'OSArchitecture',
                                                   'Processor', 'EngineVersion',
                                                   'LicenseActivationChannel',
                                                   'SKUEditionName',
                                                   'MDC2FormFactor',
                                                   'PlatformType',
                                                   'AutoUpdateOptionsName',
                                                   'OSGenuineState',
                                                   'FlightRing', 'OSBranch',
                                                   'PrimaryDiskType',
                                                   'OSInstallType',
                                                   'OsPlatformSubRelease'])]))])

In [104]:
X_train_trf = pipe.transform(X_train)
X_val_trf = pipe.transform(X_val)

#Selector

In [ ]:
from sklearn.feature_selection import SelectKBest, mutual_info_classif, f_classif
SelectKBest_selector = SelectKBest(mutual_info_classif, k=67)
SelectKBest_selector.fit(X_train_trf, y_train)

In [ ]:
X_train_selected = SelectKBest_selector.transform(X_train_trf)
X_val_selected = SelectKBest_selector.transform(X_val_trf)

In [ ]:
def selector_method(selector, X_train_trf, y_train):
    selector.fit(X_train_trf, y_train)
    X_train_selected = selector.transform(X_train_trf)
    X_val_selected = selector.transform(X_val_trf)
    return X_train_selected, X_val_selected

# DummyClassifier

In [ ]:
# baseline model
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

# Dummy Classifier with default strategy
dummy_clf = DummyClassifier(random_state=42)
dummy_clf.fit(X_train, y_train)

# Compute accuracy for training data
train_accuracy = accuracy_score(y_train, dummy_clf.predict(X_train))
print(f"Baseline Model Train Accuracy: {train_accuracy}")

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

log_reg = LogisticRegression()
log_reg.fit(X_train_selected, y_train)

y_pred_log = log_reg.predict(X_val_selected)
log_accuracy = accuracy_score(y_val, y_pred_log)
print(f"Logistic Regression Accuracy: {log_accuracy:.4f}")


In [ ]:
from sklearn.linear_model import RidgeClassifier

ridge_clf = RidgeClassifier(alpha=1.0)
ridge_clf.fit(X_train_selected, y_train)

y_pred_ridge = ridge_clf.predict(X_val_selected)
ridge_accuracy = accuracy_score(y_val, y_pred_ridge)
print(f"Ridge Classifier Accuracy: {ridge_accuracy:.4f}")


In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(n_estimators=150, random_state=42)
rf_clf.fit(X_train_selected, y_train)

y_pred = rf_clf.predict(X_val_selected)
print(f"Random Forest Accuracy: {accuracy_score(y_val, y_pred):.4f}")


In [ ]:
from xgboost import XGBClassifier

#xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf = XGBClassifier(
    n_estimators=500,  # More trees
    learning_rate=0.05,  # Lower learning rate
    max_depth=6,  # Increase depth
    min_child_weight=3,  # Reduce overfitting
    colsample_bytree=0.8,  # Column sampling
    subsample=0.8,  # Row sampling
    reg_lambda=1,  # L2 regularization
    reg_alpha=0,  # L1 regularization
    use_label_encoder=False,
    eval_metric='logloss'
)

xgb_clf.fit(X_train_selected, y_train)


y_pred = xgb_clf.predict(X_val_selected)
print(f"XGBoost Accuracy: {accuracy_score(y_val, y_pred):.4f}")


In [ ]:
from lightgbm import LGBMClassifier

lgbm_clf = LGBMClassifier()
lgbm_clf.fit(X_train_selected, y_train)

y_pred = lgbm_clf.predict(X_val_selected)
print(f"LightGBM Accuracy: {accuracy_score(y_val, y_pred):.4f}")


In [ ]:
from lightgbm import LGBMClassifier, early_stopping

lgbm_clf = LGBMClassifier(
    n_estimators=1000,         # Increase trees (more boosting rounds)
    learning_rate=0.05,        # Lower learning rate for better generalization
    max_depth=7,               # Increase depth for learning more patterns
    num_leaves=50,             # More leaves per tree
    min_child_samples=30,      # Minimum data in a leaf (avoid overfitting)
    subsample=0.8,             # Row sampling for diversity
    colsample_bytree=0.8,      # Feature sampling for diversity
    reg_alpha=0.1,             # L1 regularization (reduce overfitting)
    reg_lambda=0.1,            # L2 regularization (reduce overfitting)
    importance_type='gain'     # Feature importance based on gain
)

# Use callbacks for early stopping
lgbm_clf.fit(
    X_train_selected, y_train, 
    eval_set=[(X_val_selected, y_val)],
    callbacks=[early_stopping(stopping_rounds=50, verbose=True)]
)

y_pred = lgbm_clf.predict(X_val_selected)
print(f"LightGBM Accuracy: {accuracy_score(y_val, y_pred):.4f}")


# Test Data



In [ ]:
test = pd.read_csv("/kaggle/input/System-Threat-Forecaster/test.csv")


In [ ]:
test['ProductName'] = test['ProductName'].replace({'win8defender': 1, 'mse': 0})
test['DeviceFamily'] = test['DeviceFamily'].replace({'Windows.Server':0, 'Windows.Desktop':1})

In [ ]:
columns_to_fill = ['CityID', 'NumAntivirusProductsEnabled']
test[columns_to_fill] = test[columns_to_fill].fillna(0)   

In [ ]:
test['PrimaryDiskType'] = test['PrimaryDiskType'].replace(['UNKNOWN','Unknown','Unspecified'], 'Unknown')
test['PrimaryDiskType'] = test['PrimaryDiskType'].fillna('Unknown')

In [ ]:
test = test.drop(columns= ['IsBetaUser', 'IsFlightsDisabled','DateOS','FirmwareVersionID' , 'SystemVolumeCapacityMB' , 'OEMModelID' , 'CityID' , 'MachineID'], errors='ignore')

In [ ]:
numeric_cols = test.select_dtypes(include=['int']).columns.tolist()
object_cols = test.select_dtypes(include=['object']).columns.tolist()
float_cols = test.select_dtypes(include=['float64']).columns.tolist()

In [ ]:
print("\nNumeric:", len(numeric_cols), numeric_cols)
print("\nObject:", len(object_cols), object_cols)
print("\nFloat:", len(float_cols), float_cols)

In [ ]:
test = test.replace({pd.NA: np.nan, '': np.nan, None: np.nan})
all_columns = {"numeric": numeric_cols, "object": object_cols, "float": float_cols}
mode_imputer = SimpleImputer(strategy='most_frequent')   # Impute each column type

In [ ]:
for col_type, columns in all_columns.items():
    if columns:  # Ensure there are columns to process
        test[columns] = pd.DataFrame(
            mode_imputer.fit_transform(test[columns]),
            columns=columns,
            index=test.index
        )

In [ ]:
# Selected numerical and categorical features based on correlation analysis
numerical_col = [
    'NumAntivirusProductsInstalled', 'AntivirusConfigID','AutoSampleSubmissionEnabled',
    'TotalPhysicalRAMMB', 'IsAlwaysOnAlwaysConnectedCapable', 'IsSystemProtected',
    'IsGamer', 'ProcessorCoreCount', 'PrimaryDiskCapacityMB', 'RealTimeProtectionState',
    'IsTouchEnabled', 'IsPassiveModeEnabled', 'IsVirtualDevice', 'NumAntivirusProductsEnabled',
    'OSBuildNumberOnly', 'PrimaryDisplayDiagonalInches', 'OSBuildNumber',
    'PrimaryDisplayResolutionHorizontal', 'IEVersionID', 'HasOpticalDiskDrive',
    'ProcessorModelID', 'OSProductSuite', 'IsPenCapable', 'FirmwareManufacturerID',
    'RegionIdentifier', 'InternalBatteryNumberOfCharges', 'LocaleEnglishNameID',
    'SMode', 'OEMNameID', 'PrimaryDisplayResolutionVertical', 'FirewallEnabled',
    'OSBuildRevisionOnly', 'DeviceFamily', 'HasTpm', 'IsPortableOS', 'ProductName',
    'CountryID', 'OSInstallLanguageID', 'ProcessorManufacturerID', 'GeoRegionID',
    'IsSecureBootEnabled', 'OSUILocaleID', 'EnableLUA'
]


categorical_col = [
    'OSVersion', 'SignatureVersion', 'OSEdition', 'OSBuildLab','DateAS',
    'OSSkuFriendlyName', 'PowerPlatformRole', 'AppVersion', 'NumericOSVersion',
    'ChassisType', 'OSArchitecture', 'Processor', 'EngineVersion',
    'LicenseActivationChannel', 'SKUEditionName', 'MDC2FormFactor',
    'PlatformType', 'AutoUpdateOptionsName', 'OSGenuineState', 'FlightRing',
    'OSBranch', 'PrimaryDiskType', 'OSInstallType', 'OsPlatformSubRelease'
]


In [ ]:
from sklearn.impute import SimpleImputer

# Step 1: Select only the specified columns from the test data
selected_col = numerical_col + categorical_col
test = test[selected_col]

# Step 2: Impute missing values in the selected columns
mode_imputer = SimpleImputer(strategy='most_frequent')

# Apply imputation
test[selected_col] = pd.DataFrame(
    mode_imputer.fit_transform(test[selected_col]),
    columns=selected_col,
    index=test.index
)

# Ensure proper data types after imputation
for col in numerical_col:
    test[col] = test[col].astype(float)  # Convert numerical columns to float
for col in categorical_col:
    test[col] = test[col].astype(str)    # Convert categorical columns back to string

# Step 3: Transform the test data using the preprocessing pipeline
test_transform = pipe.transform(test)

# Step 4: Apply PCA transformation to reduce dimensions
test_transform_selector = SelectKBest_selector.transform(test_transform)

# Step 5: Make predictions using the best model
y_pred = xgb_clf.predict(test_transform_selector)

# Step 6: If needed, map predictions back to original labels
# Example: y_pred_mapped = pd.Series(y_pred).map({0: 'no', 1: 'yes'})

# Step 7: Create a DataFrame for submission or further evaluation
prediction_df = pd.DataFrame({
    'id': test.index,   # Ensure you have an 'id' column or relevant index
    'target': y_pred       # Replace with y_pred_mapped if label mapping is applied
})

# Save the predictions to a CSV file for submission or analysis
prediction_df.to_csv('submission.csv', index=False)

# Print a sample of predictions
print(prediction_df.head())

In [ ]:
prediction_df = pd.DataFrame({
    'id': test.index,
    'target': y_pred
})
prediction_df.to_csv('/kaggle/working/submission.csv', index=False)